In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset


# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# device 및 torch_dtype 설정 (mps 지원)
if torch.cuda.is_available():
    device = "cuda:0"
    torch_dtype = torch.float16
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
    torch_dtype = torch.float16  # 오류 발생 시 float32로 변경
else:
    device = "cpu"
    torch_dtype = torch.float32

print(f"device: {device}, torch_dtype: {torch_dtype}")

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
    chunk_length_s=10,
    stride_length_s=2,
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = "./mp3/lsy_audio_2023_58s.mp3"

result = pipe(sample)
# print(result["text"])
print(result)


device: mps, torch_dtype: torch.float16


Device set to use mps
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
/Users/ng/Documents/workspace/ai/llm-ai-agent/.venv/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:604: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


{'text': ' 안녕하세요. 이 강의는 GPT-API로 챗봇 만들기 라는 내용을 다루는 강의입니다. GPT-API에 대해서 생소하신 분들도 있을텐데 우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서 우리가 원하는 프로그램을 어떻게 만드는지에 대해서 이야기할 거예요. 그래서 이런 강의들이 사실 많이 있습니다. 그래서 여러가지들이 있는데 이 강의 특징이라고 한다면 GPT로 명확한 미션을 달성하는 챕터 프로그램을 만드는게 사실 쉽지는 않은데 이걸 어떻게 해서 구현을 하는지 그리고 그게 왜 필요한지에 대해서 좀 이야기를 할 거고요. 그 예제로 예제는 여러가지가 될 수 있는데 예제로 하는 것은 음악 플레이리스트 동영상을 자동으로 대화를 통해서 생성하는 프로그램을 만드는 것을 다루려고 합니다. 프로그램이 실행되는 모습을 한번 보여드릴게요. 우리가 만들 프로그램은 이런 식으로 이제 나타나게 되고', 'chunks': [{'timestamp': (0.0, 6.3), 'text': ' 안녕하세요. 이 강의는 GPT-API로 챗봇 만들기 라는 내용을 다루는 강의입니다.'}, {'timestamp': (7.18, 10.0), 'text': ' GPT-API에 대해서 생소하신 분들도 있을텐데'}, {'timestamp': (11.0, 17.0), 'text': ' 우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서'}, {'timestamp': (17.0, 20.0), 'text': ' 우리가 원하는 프로그램을 어떻게 만드는지에 대해서'}, {'timestamp': (20.0, 22.0), 'text': ' 이야기할 거예요.'}, {'timestamp': (22.0, 24.0), 'text': ' 그래서 이런 강의들이 사실 많이 있습니다.'}, {'timestamp': (24.0, 27.48), 'text': ' 그래서 여러가지들이 있는데 이 강의 특징이라고 한다면'}, {'timestamp': (27.48, 29.58), 'text': ' G

In [3]:
# CSV로 저장
start_end_text = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start, end, text])

print(start_end_text)

import pandas as pd

df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("./output/lsy_audio_2023_58s.csv", index=False, sep="|")
display(df)

[[0.0, 6.3, ' 안녕하세요. 이 강의는 GPT-API로 챗봇 만들기 라는 내용을 다루는 강의입니다.'], [7.18, 10.0, ' GPT-API에 대해서 생소하신 분들도 있을텐데'], [11.0, 17.0, ' 우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서'], [17.0, 20.0, ' 우리가 원하는 프로그램을 어떻게 만드는지에 대해서'], [20.0, 22.0, ' 이야기할 거예요.'], [22.0, 24.0, ' 그래서 이런 강의들이 사실 많이 있습니다.'], [24.0, 27.48, ' 그래서 여러가지들이 있는데 이 강의 특징이라고 한다면'], [27.48, 29.58, ' GPT로 명확한 미션을 달성하는'], [29.58, 31.66, ' 챕터 프로그램을 만드는게 사실'], [31.66, 34.32, ' 쉽지는 않은데 이걸 어떻게 해서'], [34.32, 36.4, ' 구현을 하는지 그리고 그게 왜 필요한지에 대해서'], [36.4, 37.34, ' 좀 이야기를 할 거고요.'], [38.0, 40.0, ' 그 예제로'], [40.0, 42.0, ' 예제는 여러가지가 될 수 있는데'], [42.0, 44.2, ' 예제로 하는 것은 음악 플레이리스트 동영상을'], [44.2, 46.04, ' 자동으로 대화를 통해서 생성하는'], [46.04, 48.46, ' 프로그램을 만드는 것을 다루려고 합니다.'], [49.84, 51.96, ' 프로그램이 실행되는 모습을 한번 보여드릴게요.'], [52.84, 58.0, ' 우리가 만들 프로그램은 이런 식으로 이제 나타나게 되고']]


start    end                                             text
0    0.00   6.30   안녕하세요. 이 강의는 GPT-API로 챗봇 만들기 라는 내용을 다루는 강의입니다.
1    7.18  10.00                       GPT-API에 대해서 생소하신 분들도 있을텐데
2   11.00  17.00            우리가 잘 알고 있는 ChatGPT, ChatGPT 기능을 이용해서
3   17.00  20.00                      우리가 원하는 프로그램을 어떻게 만드는지에 대해서
4   20.00  22.00                                        이야기할 거예요.
5   22.00  24.00                          그래서 이런 강의들이 사실 많이 있습니다.
6   24.00  27.48                    그래서 여러가지들이 있는데 이 강의 특징이라고 한다면
7   27.48  29.58                                GPT로 명확한 미션을 달성하는
8   29.58  31.66                                 챕터 프로그램을 만드는게 사실
9   31.66  34.32                                쉽지는 않은데 이걸 어떻게 해서
10  34.32  36.40                       구현을 하는지 그리고 그게 왜 필요한지에 대해서
11  36.40  37.34                                    좀 이야기를 할 거고요.
12  38.00  40.00                                            그 예제로
13  40.00  42.00                                예제는 여러가지가 될 수 있는데
14  42.00  44.20                         예제로 하는 것은 음악 플레이리스트 동영상을
15  44.20  46.04                                자동으로 대화를 통해서 생성하는
16  46.04  48.46                           프로그램을 만드는 것을 다루려고 합니다.
17  49.84  51.96                        프로그램이 실행되는 모습을 한번 보여드릴게요.
18  52.84  58.00                   우리가 만들 프로그램은 이런 식으로 이제 나타나게 되고